# Analysis of ambiental radioactivity ntuples by Matthew Green
In this notebook, we analyse the events contained in the root files by Matt Green about ambiental radioactivity.

TODO list:
1. adapt python script to ntuple output scheme - **Done**
1. run the script to extract csv from root - **Done**
1. start analysis

Note:
1. The csv files are huge, in the order of 1-2gb. I will try to split in smaller files ordered by event number.

## First file: A228_Z90
Explaination of the name: A=228, Z=90 is **228Th**, an isotope of Thorium.
Then, take a look at the atom involves in the simulations.

In [5]:
import pandas as pd
import os

filepath = os.path.join("..", "Data", "output_AmbRad_A228_Z90_100000_part1.csv")
df = pd.read_csv(filepath, index_col=False)

In [11]:
df.PID.unique()

array([1000020040,         22,         11, 1000180400,        -11,
       1000320760, 1000320740, 1000180360, 1000180380])

These are:
- `PID=-11` positrons
- `PID=11` electrons
- `PID=22` photons
- `PID=1000020040` Helium isotope 4He
- `PID=1000180360` Argon isotope 36Ar
- `PID=1000180380` Argon isotope 38Ar
- `PID=1000180400` Argon isotope 40Ar
- `PID=1000320740` Germanium isotope 74Ge
- `PID=1000320760` Germanium isotope 76Ge